# Amazon SageMaker Studio を一通り試す
_**Gradient Boosted Trees を使って携帯電話の顧客離反を予測しよう**_

---

このノートブックでは、Amazon SageMaker の主な機能のうちいくつかを試すことができます。


* [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html)
  * 複数の機械学習の試行を管理する
  * ハイパーパラメータに関する実験とグラフ化
* [Amazon SageMaker Debugger](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html)
  * モデルのデバッグ
* [Model hosting](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html)
  * モデルから予測を得るための永続的なエンドポイントをセットアップ
* [SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)
  * モデルの品質を監視する
  * モデルの品質に変化があった場合にアラートをあげる
 

---

## コンテンツ

1. [背景](#背景) - XGBoost による顧客離反予測について
1. [データ](#データ) - データセットを用意して S3 にアップロード
1. [学習](#学習) - Amazon SageMaker の XGBoost を利用して学習する
  - [Amazon SageMaker Experiments](#Amazon-SageMaker-Experiments)
  - [Amazon SageMaker Debugger](#Amazon-SageMaker-Debugger)
1. [ホスティング](#ホスティング)
1. [SageMaker Model Monitor](#SageMaker-Model-Monitor)

---

## 背景

_このノートブックは [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)　を参考に作成しています。_.

どのようなビジネスであっても、顧客を失うことは大きな損害です。もし、満足していない顧客を早期に見つけることができれば、そのような顧客をキープするためのインセンティブを提供できる可能性があるでしょう。このノートブックでは、満足していない顧客を自動で認識するために機械学習 (Machine Learning, ML) を利用する方法を説明します。そして、実験管理、モデル学習、デプロイ後のモデルの監視など、SageMaker の様々な機能を利用しています。

まず、このノートブックを実行するために必要な Python ライブラリをインポートしましょう。

In [ ]:
import sys
!{sys.executable} -m pip install sagemaker -U
!{sys.executable} -m pip install sagemaker-experiments

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re


import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [ ]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

---
## データ

携帯電話会社は、どの顧客が最終的に離反したか、または、サービスを使い続けたかの履歴データをもっています。この履歴データに対して学習を行うことで、携帯電話会社の顧客離反を予想するモデルを構築します。モデルの学習が終わった後、任意の顧客のデータ (モデルの学習で利用したものと同じ情報を利用します）をモデルに入力すると、モデルはその顧客が離反しそうかどうかを予測します。

ここで利用するデータセットは一般的に利用可能で、書籍 [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) の中で Daniel T. Larose が言及しているものです。そのデータセットは、著者によって University of California Irvine Repository of Machine Learning Datasets に提供されています。ここでは、そのデーセットをダウンロードして読み込んでみます。

このノートブックと同じ階層にある `data` フォルダの中に、ダウンロードして処理済みのデータセットが保存されています。そして、学習セットとバリデーションセットに分割されています。データセットの前処理については、[XGBoost customer churn notebook that starts with the original dataset](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb) をごらんください。

学習はヘッダなしのcsvファイルを使って行いますが、以下のセルではヘッダ付きの学習データを`pandas`でロードしてみます。このデータセットがどういった特徴を持っていて、モデルの学習にどのようなデータが利用されるかを調べてみます。


In [ ]:
# Set the path we can find the data files that go with this notebook
%cd /root/amazon-sagemaker-examples/aws_sagemaker_studio/getting_started
local_data_path = './data/training-dataset-with-header.csv'
data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

ここで、学習用にS3にファイルをアップロードしますが、その前に、そのデータを保存するためのS3バケットがなければ作成しましょう。


In [ ]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'xgboost-churn'

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload('data/train.csv', 's3://{}/{}/{}'.format(bucket, prefix,'train'))
print(s3url)
s3url = S3Uploader.upload('data/validation.csv', 's3://{}/{}/{}'.format(bucket, prefix,'validation'))
print(s3url)

---
## 学習

学習に進みましょう! XGBoostというライブラリを使って、さきほどアップロードしたデータに対して、勾配ブースティング木 (Gradient Boosted Decision Trees)のモデルを学習します。XGBoostを使うにあたって、XGBoostアルゴリズムのコンテナの位置を最初に指定する必要があります。

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='0.90-2')

CSV のファイルを学習に利用するので、学習のための関数が、そのファイルがどこにあって、どういう形式なのかを`s3_input`を利用して指定します。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

### Amazon SageMaker Experiments

SageMakerの実験管理を使うことで、モデルの学習をトラックして、関連するモデルを構造的に管理し、モデルの構成、パラメータ、メトリクスのログを取り続けることができます。これによって、以前のモデルを再現したり、改善を加えたり、モデルの比較を行うことができます。ここでは、モデルを学習するための異なるアプローチについてトラックするために1つの実験を作成します。

これから動かすアプローチや学習コードは1つの実験の試行となり、SageMaker Studioでは、これらの異なる試行を比較することができます。

実験をここで作成してみましょう。

In [ ]:
sess = sagemaker.session.Session()

create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
customer_churn_experiment = Experiment.create(experiment_name="customer-churn-prediction-xgboost-{}".format(create_date), 
                                              description="Using xgboost to predict customer churn", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

#### ハイパーパラメータ

XGBoost に対してハイパーパラメータを設定することができます。重要なハイパーパラメータは以下のとおりです。

- `max_depth` は、アルゴリズムがどこまで深い木を作成するかを決めます。より深い木はデータに対してより適合できますが、計算時間も長く過学習となる可能性があります。モデルの性能を改善するためには、浅い木をたくさん作成するか、深い木を少数作成するかというトレードオフに向き合う必要があります。
- `subsample` は学習データのサンプリングをコントロールします。過学習を防ぐことができますが、あまり小さすぎる値を設定するとデータ不足に陥ります。
- `num_round` はブースティングの回数をコントロールします。これまでの残差を使って学習されるモデルの数と本質的に同じです。多くの回数を実行すると学習により適しますが、計算時間画像化し、過学習に陥る可能性があります。
- `eta` は各ブースティングがどの程度学習に反映されるかをコントロールします。より大きな値を設定すると、重みが小さくなり、保守的なブースティングになります。
- `gamma` 木の大きさをコントロールします。より大きい値を設定すると保守的なブースティングになります。
- `min_child_weight` も木の大きさをコントロールします。より大きい値を設定すると保守的なブースティングになります。

詳細は [XGBoost's hyperparameters GitHub page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md)にて確認いただけます。

In [ ]:
hyperparams = {"max_depth":5,
               "subsample":0.8,
               "num_round":600,
               "eta":0.2,
               "gamma":4,
               "min_child_weight":6,
               "silent":0,
               "objective":'binary:logistic'}

#### 試行 1 - アルゴリズムモードで XGBoost を利用する

最初の試行では、ビルトインのXGBoostのコンテナを利用し、追加のコードを作成することなくモデルを学習します。この方法ではXGBoostを使ってモデルを学習し、他のビルトインアルゴリズムと同様にモデルのデプロイも可能です。

このために`Trial`オブジェクトを作成し、そのTrialを先ほど作成した実験に関連付けます。モデルを学習するためにestimatorを作成し、使いたい学習インスタンスのタイプやその数、どこにモデルを保存するかといったパラメータを指定します。

学習ジョブと、先ほど作成した実験の試行は `estimator.fit`を呼ぶときに関連付けられます。

In [ ]:
trial = Trial.create(trial_name="algorithm-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=customer_churn_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))

xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    hyperparameters=hyperparams,
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    base_job_name="demo-xgboost-customer-churn",
                                    sagemaker_session=sess)

xgb.fit({'train': s3_input_train,
         'validation': s3_input_validation}, 
        experiment_config={
            "ExperimentName": customer_churn_experiment.experiment_name, 
            "TrialName": trial.trial_name,
            "TrialComponentDisplayName": "Training",
        }
       )

#### 結果を確認する

学習ジョブが成功したら、SageMaker Studio の実験タブの中に、試行に関連するメトリクス、ログ、グラフを見ることができます。これらを見るために実験ボタンをクリックしてください。

![experiments_button](./images/studio_experiments_icon.png)

この実験のリストから、特定の実験をダブルクリックすると、その内容を見ることができます。複数の実験の内容を見たい場合は、controlをおしながらクリックしてそれらを選び、右クリックでコンテクストメニューを見ることができます。そこから、 "Open in trial component list" をクリックすることで、一緒に内容を見ることができます。これによって異なる実験を比較するようなグラフ化も可能です。

![studio_experiments_list](./images/view_multiple_trial_components.gif)

それぞれの内容は、最善のモデルが一番上に表示されるようにソートされています。

![studio_trial_list](./images/studio_trial_list.png)

#### モデルのダウンロード

生成されたモデルを見つけてダウンロードすることができます。モデルを見つけるためには、左側のトレイの実験ボタンをクリックし、実験、最近の試行のリスト、それらの最近の内容をクリックしていくと、Describe Trial Componentsの画面が見えます。これには Artifacts のタブがあります。このタブでは"Input Artifacts"のところに学習データとバリデーションデータへのリンクが、そして "Output Artifacts"のところには生成されたモデルへのリンクが表示されています。

![find the model artifact](./images/find_the_model_artifact.gif)

#### 他のハイパーパラメータを試す

モデルを改善するためによく実施される方法としては、他のハイパーパラメータを試し、それらが最終的なバリデーションエラーに影響を与えるかどうかを見ることです。ここでは、`min_child_weight`のパラメータを変えて、それらの値を変えながら学習ジョブを実行し、バリデーションエラーが変わるかどうかを見ます。それぞれに対して、1つ1つ試行を作成することで、SageMaker Studioで結果を比較することができます。


In [ ]:
min_child_weights = [1, 2, 4, 8, 10]

for weight in min_child_weights:
    hyperparams["min_child_weight"] = weight
    trial = Trial.create(trial_name="algorithm-mode-trial-{}-weight-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()), weight), 
                         experiment_name=customer_churn_experiment.experiment_name,
                         sagemaker_boto_client=boto3.client('sagemaker'))

    t_xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                          role=role,
                                          hyperparameters=hyperparams,
                                          train_instance_count=1, 
                                          train_instance_type='ml.m4.xlarge',
                                          output_path='s3://{}/{}/output'.format(bucket, prefix),
                                          base_job_name="demo-xgboost-customer-churn",
                                          sagemaker_session=sess)

    t_xgb.fit({'train': s3_input_train,
               'validation': s3_input_validation},
                wait=False,
                experiment_config={
                    "ExperimentName": customer_churn_experiment.experiment_name, 
                    "TrialName": trial.trial_name,
                    "TrialComponentDisplayName": "Training",
                }
               )

#### グラフの作り方

試行の内容を複数選んでグラフを作ることができます。これは1回の学習を試しただけなので、わずかなデータしかなく、時系列でグラフ化するほどではないでしょう。しかし、パラメータの変化を散布図でプロットすることはできます。その例を以下の画像で示します。

![scatter plot example](./images/scatter_plot_example.png)

##### 散布図の作り方

内容の複数選択を行い、"Add chart"をクリックします。Chart Properties のパネルで、データタイプとして"Summary Statistics"を選びます。グラフのタイプは scatter plot (散布図) を選びます。そして、`min_child_wight`をX軸に (これはノートブックで反復的に変化させた値です)、`validation:error_last`や `validation:error_avg` をY軸のメトリクスとして選択して、`trialComponentName`で色をつけます。

![create a scatter plot](./images/create_a_scatter_plot.gif)

選択する内容を変えることによって、その場でグラフを変更することも可能ですし、拡大縮小を行うこともできます。グラフ上に各項目を表示することで、各試行に対する情報を得ることができるでしょう。

![adjust a scatter plot](./images/adjust_a_scatter_plot.gif)

### Amazon SageMaker Debugger

Amazon SageMaker は学習時の機械学習モデルのデバッグを可能にします。学習の間、デバッガーは定期的にテンソルを保存し、そのテンソルはある瞬間における機械学習モデルの状態を完全に保持します。これらのテンソルは、SageMaker Studio を使って、学習時の課題を診断するための分析や可視化のためにS3に保存される。

#### SageMaker のデバッグルールを指定する

機械学習モデルの学習時に一般的に起こる課題を自動で検出するために、SageMaker debugger では、学習ジョブを評価する一連のリストを付与することを可能にしています。

XGBoostに適用可能ないくつかのルール設定として、`AllZero`, `ClassImbalance`, `Confusion`, `LossNotDecreasing`, `Overfit`, `Overtraining`, `SimilarAcrossRuns`, `TensorVariance`, `UnchangedTensor`, `TreeDepth`があります。

ここでは、`LossNotDecreasing`という学習時のある時点でロスが低下しなくなることを検知するルールと、`Overtraining`のルール、`Overfit`のルールを適用します。これらのルールを作ってみましょう。

In [ ]:
debug_rules = [Rule.sagemaker(rule_configs.loss_not_decreasing()),
               Rule.sagemaker(rule_configs.overtraining()),
               Rule.sagemaker(rule_configs.overfit())
              ]

#### 試行2 - フレームワークモードでXGBoostを利用する

次の試行では、似たようなモデルを学習しますが、フレームワークモードでXGBoostを利用します。このXGBoostの利用方法は、オープンソースのXGBoostを利用している人にとっては親しみのある方法でしょう。フレームワークモードででXGBoostを利用することで、ビルトインアルゴリズムとして利用するより柔軟性高く利用でき、前処理や後処理のスクリプトを学習スクリプトに組むこむことでより発展的なシナリオを扱うことができます。具体的には、SageMaker Debugger が評価する学習スクリプトに対して適用される一連のルールを設定することができます。

#### Estimator に Fit を実行する

XGBoost をフレームワークモードで実行するためには、学習ジョブに追加の処理を組みこんだ エントリーポイント (entry-point) を指定する必要があります。

SageMaker Debugger `smdebug` を利用できるようにするために、いくつかの変更を加えます。Booster を作成するときに、コールバック関数を渡せるように SessionHook を作成します。Hookに対して、評価用のメトリクス、特徴重要度、SHAP値を定期的に保存するよう伝えるSaveConfig オブジェクトを渡します。Sagemaker Debugger は自由な設定が可能なので、保存したいものを厳密に指定することも可能です。これらの変更については、[Developer Guide for XGBoost](https://github.com/awslabs/sagemaker-debugger/tree/master/docs/xgboost) でも説明されているこのサンプルの学習が終わった後に、詳しく説明します。


In [ ]:
!pygmentize xgboost_customer_churn.py

Framework estimator を作成して、学習を始めるために `fit` を呼び出しましょう。以前と同様に、この実行のために別々の試行を作成することで、SageMaker Studioで後ほど結果を比較することができます。フレームワークモードで実行しているので、エントリーポイントのスクリプトや、フレームワークのバージョンなど、追加のパラメータを estimator にわたす必要があります。

学習が進むと、学習ジョブに対するルールを評価する SageMaker Debugger から、ログを見ることができるでしょう。


In [ ]:
entry_point_script = "xgboost_customer_churn.py"

trial = Trial.create(trial_name="framework-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=customer_churn_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))

framework_xgb = sagemaker.xgboost.XGBoost(image_name=docker_image_name,
                                          entry_point=entry_point_script,
                                          role=role,
                                          framework_version="0.90-2",
                                          py_version="py3",
                                          hyperparameters=hyperparams,
                                          train_instance_count=1, 
                                          train_instance_type='ml.m4.xlarge',
                                          output_path='s3://{}/{}/output'.format(bucket, prefix),
                                          base_job_name="demo-xgboost-customer-churn",
                                          sagemaker_session=sess,
                                          rules=debug_rules
                                          )

framework_xgb.fit({'train': s3_input_train,
                   'validation': s3_input_validation}, 
                  experiment_config={
                      "ExperimentName": customer_churn_experiment.experiment_name, 
                      "TrialName": trial.trial_name,
                      "TrialComponentDisplayName": "Training",
                  })

学習がしばらく実行されると、Debuggerの画面からデバッグの情報を見ることができるでしょう。この画面に移動するためには、experiment、trial、componentとクリックしていきます。

![view_debugger panel](./images/view_studio_debugger_output.gif)

---
## ホスティング

モデルの学習が終われば、そのモデルをデプロイして、ホストするエンドポイントを作成しましょう。モデルがホストされて、リクエストを受けるようになり、そのモデルを監視するためには、エンドポイントに送信されるデータをキャプチャする設定を追加します。


In [ ]:
data_capture_prefix = '{}/datacapture'.format(prefix)

endpoint_name = "demo-xgboost-customer-churn-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName = {}".format(endpoint_name))

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, 
                           instance_type='ml.m4.xlarge',
                           endpoint_name=endpoint_name,
                           data_capture_config=DataCaptureConfig(enable_capture=True,
                                                                 sampling_percentage=100,
                                                                 destination_s3_uri='s3://{}/{}'.format(bucket, data_capture_prefix)
                                                                )
                           )

### デプロイされたモデルを呼び出す

ホストされたエンドポイントを動かすことができれば、http POST のリクエストを実行することで、モデルから簡単にリアルタイムな推論を実行できます。しかしまずは、`test_data`の NumPy 配列をエンドポイントのモデルに渡すために、serializers と deserializers をセットアップします。


In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

ここでテスト用のデータセットをループで回しながら、XGBoost のエンドポイントを呼び出し、推論結果を一通り集めます。


In [ ]:
print("Sending test traffic to the endpoint {}. \nPlease wait for a minute...".format(endpoint_name))

with open('data/test_sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        response = xgb_predictor.predict(data=payload)
        time.sleep(0.5)

### S3 のキャプチャデータを確認する

エンドポイントにデータを送ることでリアルタイムな推論を実行したので、監視のために、データがキャプチャされているはずです。

S3 にあるキャプチャデータのファイルの一覧を見てみましょう。推論が行われた時間ごとに異なるファイルを見ることができるでしょう。S3パスのフォーマットは以下のようになります。


`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
print("Found Data Capture Files:")
capture_files = S3Downloader.list("s3://{}/{}".format(bucket, current_endpoint_capture_prefix))
print(capture_files)

キャプチャされたすべてのデータは SageMaker の特別な json-line でフォーマットされたファイルに保存されます。jsonl でフォーマットされた1行の中身を少し見てみると、そのフォーマットについて知ることができるでしょう。

In [ ]:
capture_file = S3Downloader.read_file(capture_files[-1])

print("=====Single Data Capture====")
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2)[:2000])

ご覧のとおり、各推論のリクエストは jsonl ファイルの一行でキャプチャされています。その行は入出力が一緒に含んでいます。この例では、ContentType を `text/csv` で与えたので、それは`observedContentType`の値に反映されています。また、 `encoding` の値からは、入出力の payload に使われたエンコーディングについて知ることができます。

これまで、新しいパラメータで、エンドポイントに送信される入出力の payload がどのようにキャプチャされるかを確認しました。キャプチャされたデータのフォーマットが S3 でどのように見えるかも確認しました。それでは、S3 に収集されたデータのモニタリングを SageMaker がどのように役に立つかを見てみましょう。

---
## SageMaker Model Monitor

データの収集に加えて、SageMaker は Endpoints で観測されたデータの監視と評価を行うことを可能にします。このために:

1. リアルタイムに送信されるデータに対して比較するベースラインを作成する
1. ベースラインを準備できれば、ベースラインに対して継続的に評価/比較するためのスケジュールをセットアップする
1. アラームを起動するために合成データを送信する

**注意**: このセクションは完了までに1時間以上かかります。ポーリングする最小時間が1時間であるためです。以下の図は、数時間実行して、合成データに対してエラーとなったあとの結果を表しています。

![model monitor example](./images/view_model_monitor_output.gif)

### ベースラインの作成と継続的な監視

#### 1. ベースライン・学習のデータセットによる制約の提案

モデルの学習に利用した学習データセットは、通常良いベースラインのデータセットとなります。学習データセットのデータスキーマと推論用データセットのデータスキーマは完全に一致している必要があります (例えば、特徴の数やタイプなど)

学習データセットを利用して、ベースラインとなる `constraints` や、データを調べるための説明付きの `statistics` の生成を SageMaker で行いましよう。この例では、モデルの学習に利用した学習データセットをアップロードします。特徴の名前を記述した列のヘッダが入ったデータセットファイルを使います。

In [ ]:
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))
baseline_data_path = S3Uploader.upload("data/training-dataset-with-header.csv", baseline_data_uri)

##### 学習データセットでベースラインジョブを作成する

学習データをS3に用意できたら、constraints を `suggest` するジョブを実行しましょう。
SageMaker で利用可能な ProcessingJob コンテナを利用して、Constraints を生成します。

In [ ]:
my_default_monitor = DefaultModelMonitor(role=role,
                                         instance_count=1,
                                         instance_type='ml.m5.xlarge',
                                         volume_size_in_gb=20,
                                         max_runtime_in_seconds=3600,
                                        )

baseline_job = my_default_monitor.suggest_baseline(baseline_dataset=baseline_data_path,
                                                   dataset_format=DatasetFormat.csv(header=True),
                                                   output_s3_uri=baseline_results_uri,
                                                   wait=True
)

ジョブが成功すると、S3にある `baseline_results_uri` の場所を探索して、どのようなファイルがそこに保存されているかを見ることができます。

In [ ]:
print("Found Files:")
S3Downloader.list("s3://{}/{}".format(bucket, baseline_results_prefix))

Suggest される contriants に関する情報をもつ `constraints.json` を見つけることができます。また、ベースラインのデータに関する統計的情報を含む `statistics.json` も見つけることができます。

In [ ]:
baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [ ]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

#### 2. データ品質の課題に対してキャプチャデータを分析する

ベース他院のデータセットを生成し、それを処理してベースラインの statistics や constraints を取得したので、Monitoring Schedules でエンドポイントに送信されるデータを監視したり、解析したりしましょう。


##### スケジュールを作成する

まずは先ほど作成したエンドポイントに対して Monitoring Schedule を作成します。ベースラインと最新のデータキャプチャを比較するための新しい processing job を実行するタイミングを指定します。


In [ ]:
# First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing
code_prefix = '{}/code'.format(prefix)
pre_processor_script = S3Uploader.upload('preprocessor.py', 's3://{}/{}'.format(bucket,code_prefix))
s3_code_postprocessor_uri = S3Uploader.upload('postprocessor.py', 's3://{}/{}'.format(bucket,code_prefix))

エンドポイントに対する Monitoring Scheduleと、ベースラインのリソース (constraints と statistics) を作成する準備ができました。

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

mon_schedule_name = 'demo-xgboost-customer-churn-model-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(monitor_schedule_name=mon_schedule_name,
                                              endpoint_input=xgb_predictor.endpoint,
                                              #record_preprocessor_script=pre_processor_script,
                                              post_analytics_processor_script=s3_code_postprocessor_uri,
                                              output_s3_uri=s3_report_path,
                                              statistics=my_default_monitor.baseline_statistics(),
                                              constraints=my_default_monitor.suggested_constraints(),
                                              schedule_cron_expression=CronExpressionGenerator.hourly(),
                                              enable_cloudwatch_metrics=True,
                                             )

#### 3. 人工的なデータを送信する

以下のブロックでは、エンドポイントに対してデータを送信するスレッドを実行します。
エンドポイントにデータを送り続けることができるので、分析のためにデータを継続的にキャプチャすることができます。もしデータ送信がなければ、monitoring のジョブは停止を始めます。

このスレッドを停止するためには、カーネルを停止する必要があります。

In [ ]:
from threading import Thread
from time import sleep
import time

runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                          ContentType='text/csv', 
                                          Body=payload)
            time.sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'data/test-dataset-input-cols.csv', runtime_client)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

##### 実行結果をリストで表示する

Schedule がスケジュールされると、一定間隔でジョブを実行します。最新の5回の実行について、ここでリストを表示しましょう。1時間おきの schedule が作成される前に、ここを実行していれば、空の結果をみることになるでしょう。実行結果を見るためには、毎時 (UTCで) を過ぎるまで待つ必要があります。以下は、時間を待つためのコードになります。


In [ ]:
mon_executions = my_default_monitor.list_executions()
if len(mon_executions) == 0:
    print("We created a hourly schedule above and it will kick off executions ON the hour.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()  

##### 最新の実行結果と生成されたレポートのリスト

In [ ]:
latest_execution = mon_executions[-1]
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))
report_uri = latest_execution.output.destination

print("Found Report Files:")
S3Downloader.list(report_uri)

##### 違反結果のリスト

もしベースラインと比較して違反があるようであれば、ここに生成されます。違反結果をリストで表示しましょう。


In [ ]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

違反結果の詳細な可視化やキャプチャデータの統計分布を見るために、モニタリングを実行した processing job の ARN をこのノートブックで呼び出して利用することも可能です。


## クリーンアップ

このノートブックを完了したら、以下のセルを実行してください。これによって、作成したエンドポイントを削除し、不要で利用されていないインスタンスからの課金を止めることができます。実験に関係する結果の削除も行われます。

また、このノートブックで使用した S3 バケットにあるオブジェクトを削除したい場合は、S3 のコンソールに移動して、`sagemaker-studio-<region-name>-<account-name>` のバケットを探し、このノートブックに関連するファイルを削除してください。

In [ ]:
sess.delete_monitoring_schedule(mon_schedule_name)
sess.delete_endpoint(xgb_predictor.endpoint)
def cleanup(experiment):
    '''Clean up everything in the given experiment object'''
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        
        for trial_comp_summary in trial.list_trial_components():
            trial_step=TrialComponent.load(trial_component_name=trial_comp_summary.trial_component_name)
            print('Starting to delete TrialComponent..' + trial_step.trial_component_name)
            sm.disassociate_trial_component(TrialComponentName=trial_step.trial_component_name, TrialName=trial.trial_name)
            trial_step.delete()
            time.sleep(1)
         
        trial.delete()
    
    experiment.delete()

cleanup(customer_churn_experiment)